# Experiment 3: Comparing NLP algorithms
## Comparing fine-tuned BERT to RNN Implementation


## Install Dependencies


In [ ]:
%pip install datasets
%pip install transformers
%pip install spacy


BELOW TAKES FOREVER!

In [ ]:
%pip install torch

In [ ]:
%pip install spacy-transformers
%pip install transformers[torch]
%pip install seqeval

In [20]:
import torch
import torchtext

SEED = 1234
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

print("PyTorch Version: ", torch.__version__)
print("torchtext Version: ", torchtext.__version__)
print(f"Using {'GPU' if str(DEVICE) == 'cuda' else 'CPU'}.")

PyTorch Version:  2.2.0+cpu
torchtext Version:  0.16.2+cpu
Using CPU.


In [21]:
from datasets import load_dataset, load_metric
dataset = load_dataset("surrey-nlp/PLOD-CW")

In [22]:
#start by fetching the training part of the dataset
training = dataset["train"]
valid = dataset["validation"]
test = dataset["test"]


# Loading in BERT

# Pre-processing data for RNN

In [23]:
from torchtext.data.functional import to_map_style_dataset

# This might take a while
train_data = to_map_style_dataset(training)
valid_data = to_map_style_dataset(valid)
test_data = to_map_style_dataset(test)

In [24]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 1072
Number of testing examples: 153


## Building a vocabulary for the RNN

In [25]:
from torchtext.data.utils import get_tokenizer

class SpacyTokenizer(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.tokenizer = get_tokenizer("spacy", language="en_core_web_sm")
    
    def forward(self, input):
        if isinstance(input, list):
            tokens = []
            for text in input:
                tokens.append(self.tokenizer(text))
            return tokens
        elif isinstance(input, str):
            return self.tokenizer(input)
        raise ValueError(f"Type {type(input)} is not supported.")

In [27]:
from torchtext.vocab import build_vocab_from_iterator, vocab
from torchtext.data.utils import get_tokenizer
from collections import OrderedDict

tokenizer = SpacyTokenizer()
MAX_VOCAB_SIZE = 25_000

def _process_texts_for_vocab(data):
    for line in data:
        yield tokenizer(line["tokens"])
        
def _get_labels_for_vocab(data):
    for line in data:
        yield [line["ner_tags"]]

# This might take a while as we're tokenizing
text_vocab = build_vocab_from_iterator(_process_texts_for_vocab(train_data), specials=('<unk>', '<pad>'), max_tokens=MAX_VOCAB_SIZE)
label_vocab = vocab(OrderedDict([("neg", 1), ("pos", 1)]))

text_vocab.set_default_index(text_vocab["<unk>"])

TypeError: unhashable type: 'list'